In [ ]:
import hdf5_getters as GETTERS
import h5py
import glob
from pyspark.sql import SparkSession
import pyspark
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

import numpy as np
import hdf5_getters as GETTERS
import h5py
import glob
import pandas as pd

In [2]:
spark = SparkSession \
    .builder \
    .appName('XML ETL') \
    .master("local[*]") \
    .config('job.local.dir', 'file:/models/music-recommender-model') \
    .getOrCreate()

df = spark.range(10).toDF("number")
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/24 06:45:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
+------+



#### Sacar CSV del DATASET

In [6]:
import glob
import numpy as np
import pandas as pd
import hdf5_getters as GETTERS

dataset_path = "/opt/bitnami/spark/datasets/"
file_paths = glob.glob(f"{dataset_path}/**/*.h5", recursive=True)
file_paths = file_paths

def extract_features(path):
    try:
        h5 = GETTERS.open_h5_file_read(path)

        features = {
            'track_id': GETTERS.get_track_id(h5).decode('utf-8'),
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist_name': GETTERS.get_artist_name(h5).decode('utf-8'),
            'year': GETTERS.get_year(h5),
            'duration': float(GETTERS.get_duration(h5)),
            'tempo': float(GETTERS.get_tempo(h5)),
            'key': int(GETTERS.get_key(h5)),
            'mode': int(GETTERS.get_mode(h5)),
            'loudness': float(GETTERS.get_loudness(h5)),
            'danceability': float(GETTERS.get_danceability(h5)),
            'energy': float(GETTERS.get_energy(h5)),
            'song_hotttnesss': float(GETTERS.get_song_hotttnesss(h5)),
            'artist_hotttnesss': float(GETTERS.get_artist_hotttnesss(h5)),
            'artist_familiarity': float(GETTERS.get_artist_familiarity(h5))
        }

        h5.close()
        return features

    except Exception as e:
        print(f"Error procesando el fichero: {path} --> {e}")
        return None

# Cargar todos los datos en un DataFrame
data = []
for path in file_paths:
    result = extract_features(path)
    if result:
        data.append(result)

df = pd.DataFrame(data)
df.to_csv("muestra_million_song_filtrada.csv", index=False)


In [1]:
import sweetviz as sv
import pandas as pd
import numpy as np

df = pd.read_csv("muestra_million_song_filtrada.csv")
reporte = sv.analyze(df)
reporte.show_html("EDA_MillionSong.html")


c:\Users\majimenez\AppData\Local\anaconda3\envs\TFM-UNIR\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report EDA_MillionSong.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [1]:
from ydata_profiling import ProfileReport
import pandas as pd

df = pd.read_csv("muestra_million_song_filtrada.csv")
profile = ProfileReport(df, title="EDA Million Song Dataset", explorative=True)
profile.to_file("EDA_MillionSong_2.html")


c:\Users\majimenez\AppData\Local\anaconda3\envs\TFM-UNIR\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 31.45it/s]
